PrepSense Stats View

Selector for Week, Month, Year

Important:

From Daniel list:
  1. Total pantry items count: count total items in pantry
  2. Expiring items ( you can take the text from point 4 in code below)
  3. Recently added items (last 7 days)
  4. Recipes cooked counts (week/month/total)
  5. Favorite recipes (based on cooking frequency)
  6. Cooking streak calculation
  
From Bonny List:
  1. Count of expired items in pantry (point 1)
  2. Top products in the pantry (point 3)
  3. Food saved from expiration (kg_saved = (unexpired_items * 0.3) where 0.3 is the average weight of houshold item which ranges from 250-500gms)
  4. co2 emmissions: co2 emmissions saved from food saved from expiry (kg_saved * 2.5) where          2.5kg is co2 emmissions from each kg of food. this can be improved to get the co2 per         food category
 
 
  
 Nice to Have:
  1. Shopping list completion rate
  2. Proportion of differerent Food Categories in the pantry (point 2)- not accurate at the moment


In [12]:
from google.cloud import bigquery
import pandas as pd
from datetime import date, timedelta

In [11]:
# Set up client
client = bigquery.Client()

# Set user/pantry ID
pantry_id = 1011 

# Today's date
today = date.today()
next_week = today + timedelta(days=7)

# 1. Count of expired items in the pantry
pantry_query = f"""
SELECT 
  COUNT(*) AS total_items,
  COUNTIF(expiration_date >= CURRENT_DATE()) AS unexpired_items,
  COUNTIF(expiration_date < CURRENT_DATE()) AS expired_items
FROM `adsp-34002-on02-prep-sense.Inventory.pantry_items`
WHERE pantry_id = {pantry_id}
"""
pantry_df = client.query(pantry_query).to_dataframe()
# Check if expired_items is 0
expired_items = pantry_df["expired_items"].iloc[0]

if expired_items == 0:
    print("🥳 Zero expired items. Your pantry deserves a standing ovation! 👏👏👏")
else:
    print(f"⚠️ You have {expired_items} expired items in your pantry.")
    
# 2. Proportion of differerent Food Categories in the pantry
category_query = f"""
SELECT category, COUNT(*) AS count
FROM `adsp-34002-on02-prep-sense.Inventory.products`
WHERE pantry_item_id IN (
  SELECT pantry_item_id FROM `adsp-34002-on02-prep-sense.Inventory.pantry_items`
  WHERE pantry_id = {pantry_id}
)
GROUP BY category
"""
cat_df = client.query(category_query).to_dataframe()

# Calculate total number of items
total_items = cat_df["count"].sum()

# Compute proportions
cat_df["proportion"] = round((cat_df["count"] / total_items) * 100, 1)

# Drop the raw count column
cat_df.drop(columns=["count"], inplace=True)

# Sort and display
cat_df = cat_df.sort_values(by="proportion", ascending=False)

print("\n📦 Pantry Personality Test Results Are In...\n")
for _, row in cat_df.iterrows():
    category = row["category"]
    proportion = row["proportion"]

    if proportion > 40:
        comment = "Basically your whole pantry."
    elif proportion > 25:
        comment = "Dominating the shelf space!"
    elif proportion > 10:
        comment = "Respectably stocked."
    else:
        comment = "Just a sprinkle."

    print(f"🧪 {category}: {proportion}% — {comment}")


# 3. Top products in the pantry
top_query = f"""
SELECT product_name, COUNT(*) as count
FROM `adsp-34002-on02-prep-sense.Inventory.products`
WHERE pantry_item_id IN (
  SELECT pantry_item_id FROM `adsp-34002-on02-prep-sense.Inventory.pantry_items`
  WHERE pantry_id = {pantry_id} AND DATE(created_at) >= DATE_SUB(CURRENT_DATE(), INTERVAL 30 DAY)
)
GROUP BY product_name
ORDER BY count DESC
LIMIT 5
"""
top_df = client.query(top_query).to_dataframe()
#print("\n🔥 Most Used Items:\n", top_df)
print("\n🎵 Pantry Top Hits This Month:\n")
for i, row in top_df.iterrows():
    rank = i + 1
    product = row["product_name"]
    count = row["count"]
    
    emojis = ["🥇", "🥈", "🥉", "🔥", "💥"]
    prefix = emojis[i] if i < len(emojis) else f"#{rank}"
    
    print(f"{prefix} {product} — {count} times")

# 4. Expiring Soon
expiry_query = f"""
SELECT product_name, expiration_date
FROM `adsp-34002-on02-prep-sense.Inventory.products` p
JOIN `adsp-34002-on02-prep-sense.Inventory.pantry_items` pi
ON p.pantry_item_id = pi.pantry_item_id
WHERE pi.pantry_id = {pantry_id}
  AND expiration_date BETWEEN CURRENT_DATE() AND DATE_ADD(CURRENT_DATE(), INTERVAL 7 DAY)
ORDER BY expiration_date
LIMIT 5
"""
expiry_df = client.query(expiry_query).to_dataframe()
#print("\n⏳ Expiring Soon:\n", expiry_df)

expiry_df = client.query(expiry_query).to_dataframe()

if not expiry_df.empty:
    print("\n⚠️ Heads up! These items are expiring in the next 7 days:\n")
    for _, row in expiry_df.iterrows():
        product = row["product_name"]
        expiry = pd.to_datetime(row["expiration_date"]).strftime("%b %d, %Y")
        print(f"⏳ {product} → expires by {expiry}")
else:
    print("\n🎉 Great news! Nothing is expiring in the next 7 days. Your pantry is under control. 👏😎")


# 5. Calculated Insights
unexpired_items = pantry_df["unexpired_items"].iloc[0]
kg_saved = round(unexpired_items * 0.3, 2)
co2_kg = round(kg_saved * 2.5, 2)
#money_saved = round(unexpired_items * 20, 2)

print("\n♻️ Calculated Impact:")
print(f"Estimated food saved from expiration: {kg_saved} kg")
print(f"Estimated CO₂ saved: {co2_kg} kg")
#print(f"Estimated money saved: ₹{money_saved}")

🥳 Zero expired items. Your pantry deserves a standing ovation! 👏👏👏

📦 Pantry Personality Test Results Are In...

🧪 Uncategorized: 68.2% — Basically your whole pantry.
🧪 Unknown: 24.4% — Respectably stocked.
🧪 Snacks: 1.8% — Just a sprinkle.
🧪 Pantry: 1.8% — Just a sprinkle.
🧪 Dairy: 0.9% — Just a sprinkle.
🧪 Meat: 0.5% — Just a sprinkle.
🧪 Produce: 0.5% — Just a sprinkle.
🧪 Legumes: 0.5% — Just a sprinkle.
🧪 Fats & Oils: 0.5% — Just a sprinkle.
🧪 Grains: 0.5% — Just a sprinkle.
🧪 Canned Goods: 0.5% — Just a sprinkle.

🎵 Pantry Top Hits This Month:

🥇 Red Apple — 60 times
🥈 Colavita Extra Virgin Olive Oil — 39 times
🥉 Yellow Bell Pepper — 29 times
🔥 Corn on the Cob — 25 times
💥 Broccoli — 13 times

🎉 Great news! Nothing is expiring in the next 7 days. Your pantry is under control. 👏😎

♻️ Calculated Impact:
Estimated food saved from expiration: 66.6 kg
Estimated CO₂ saved: 166.5 kg
